In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..')))
from src import *
from itertools import product

import numpy as np
from matplotlib import pyplot as plt
from numba import njit
import scipy.stats as stats

In [2]:
#@njit
def centered_probabilities(states):
    half_L = states.shape[1] // 2
    out = np.full_like(states, 0.0, dtype = np.float64)
    for i, state in enumerate(states):
        probs = np.abs(state, dtype = np.float64)
        centre = np.argmax(probs)
        out[i, :] = np.roll(probs, half_L-centre)
    return out

In [3]:
def produce_data(p, L, realizations):
    data = np.zeros(L)
    for _ in range(realizations):
        ness_correlation = C_NESS(0, 0, A(L, p))
        _, eigenvectors = np.linalg.eig(ness_correlation)
        centered_probs = centered_probabilities(eigenvectors.T)
        data += np.mean(centered_probs, axis = 0)
    return data / realizations

In [4]:
def fit_log_log(x, y, subset):
    sort_permutation = np.argsort(x)
    x_data = np.log(x[sort_permutation][subset])
    y_data = np.log(y[sort_permutation][subset])
    res = stats.linregress(x_data, y_data)
    return res[0], res[1]

In [13]:
L = 1000
ps = [0.5, 0.7, 0.9, 1.0, 1.2, 1.4, 1.7, 2.0, 3.0, 5.0]
realizations = 100

In [14]:
data = list(map(lambda p: produce_data(p, L, realizations), ps))

In [16]:
%matplotlib qt

fig, ax = plt.subplots(nrows=1, ncols=2)
grads = np.zeros(len(data))
for i, y in enumerate(data):
    x = np.abs(np.arange(L) - L/2)
    g, c = fit_log_log(x, y, np.arange(20, 800))
    grads[i] = -g
    ax[0].plot(x, y, linewidth=2.5, label=r"$p = {}, a = {}, c = {}$".format(
        ps[i], round(-g, 3), round(math.exp(c)), 3))
    ax[0].plot(np.arange(10, 400), math.exp(c)*(np.arange(10, 400)**g))
ax[0].set_xlabel(r"$|i-i_\mathrm{max}|$")
ax[0].set_ylabel(r"$\left\langle|\psi(i-i_\mathrm{max})|^2\right\rangle$")
ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].set_title(r'$L = 1000, \mathrm{realizations} = 100, y(x) \approx cx^a$')
ax[0].legend()

ax[1].plot(ps, grads, label=r"$y(p)=a(p)$")
ax[1].plot(ps, ps, label=r"$y(p)=p$")
ax[1].legend()

plt.show()
